# Exposicion Visualización
Victor Angel Martínez Vidaurri A01665456

Uziel Heredia Estrada A01667072

Alan Luna Hernandez

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Cargar datos desde el Excel
ruta_archivo = "DATA/OECD,DF_BLI,+all.xlsx"
datos = pd.read_excel(ruta_archivo)

# Filtrar solo las filas donde 'Measure' es 'Value'
datos_valores = datos[datos["Measure"] == "Value"]

# Calcular el promedio por país e indicador
agrupados = datos_valores.groupby(["Country", "Indicator"])["OBS_VALUE"].mean().reset_index()

# Crear tabla pivote para tener países como filas y los indicadores como columnas
tabla = agrupados.pivot(index="Country", columns="Indicator", values="OBS_VALUE")

# Normalizar los valores entre 0 y 10
normalizador = MinMaxScaler(feature_range=(0, 10))
tabla_normalizada = pd.DataFrame(
    normalizador.fit_transform(tabla),
    index=tabla.index,
    columns=tabla.columns
)

# Invertir escala de indicadores donde valores altos son negativos
indicadores_negativos = [
    "Air pollution",
    "Homicide rate",
    "Employees working very long hours",
    "Labour market insecurity"
]

for indicador in indicadores_negativos:
    if indicador in tabla_normalizada:
        tabla_normalizada[indicador] = 10 - tabla_normalizada[indicador]

# Calcular promedios por categoría
tabla_normalizada["Housing (Prom)"] = tabla_normalizada[[
    "Housing expenditure",
    "Dwellings without basic facilities",
    "Rooms per person"
]].mean(axis=1)

tabla_normalizada["Income (Prom)"] = tabla_normalizada[[
    "Household net wealth",
    "Household net adjusted disposable income"
]].mean(axis=1)

tabla_normalizada["Jobs (Prom)"] = tabla_normalizada[[
    "Employment rate",
    "Personal earnings",
    "Long-term unemployment rate",
    "Labour market insecurity"
]].mean(axis=1)

tabla_normalizada["Community (Prom)"] = tabla_normalizada["Quality of support network"]

tabla_normalizada["Education (Prom)"] = tabla_normalizada[[
    "Years in education",
    "Student skills",
    "Educational attainment"
]].mean(axis=1)

tabla_normalizada["Environment (Prom)"] = tabla_normalizada[[
    "Water quality",
    "Air pollution"
]].mean(axis=1)

tabla_normalizada["Civic Engagement (Prom)"] = tabla_normalizada[[
    "Stakeholder engagement for developing regulations",
    "Voter turnout"
]].mean(axis=1)

tabla_normalizada["Health (Prom)"] = tabla_normalizada[[
    "Self-reported health",
    "Life expectancy"
]].mean(axis=1)

tabla_normalizada["Life Satisfaction (Prom)"] = tabla_normalizada["Life satisfaction"]

tabla_normalizada["Safety (Prom)"] = tabla_normalizada[[
    "Homicide rate",
    "Feeling safe walking alone at night"
]].mean(axis=1)

tabla_normalizada["Work-Life Balance (Prom)"] = tabla_normalizada[[
    "Time devoted to leisure and personal care",
    "Employees working very long hours"
]].mean(axis=1)

# Calcular el índice final del bienestar (Better Life Index)
categorias = [
    "Housing (Prom)",
    "Income (Prom)",
    "Jobs (Prom)",
    "Community (Prom)",
    "Education (Prom)",
    "Environment (Prom)",
    "Civic Engagement (Prom)",
    "Health (Prom)",
    "Life Satisfaction (Prom)",
    "Safety (Prom)",
    "Work-Life Balance (Prom)"
]

tabla_normalizada["Índice de Bienestar"] = tabla_normalizada[categorias].mean(axis=1)

# Ordenar países de mayor a menor bienestar
ranking = tabla_normalizada[["Índice de Bienestar"]].sort_values(by="Índice de Bienestar", ascending=False)

# Mostrar los 10 países con mejor índice de bienestar
print("TOP 10 PAÍSES CON MAYOR BIENESTAR:")
print(ranking.head(10))


TOP 10 PAÍSES CON MAYOR BIENESTAR:
Indicator    Índice de Bienestar
Country                         
Sweden                  7.395241
Iceland                 7.373135
Finland                 7.343810
Switzerland             7.319435
Norway                  7.295215
Denmark                 7.205260
New Zealand             7.117567
Netherlands             7.106115
Luxembourg              7.085617
Canada                  7.000123


## Grafica Vidaurri

In [7]:
import plotly.express as px

# Asegurar que el nombre del país esté como columna
tabla_normalizada["País"] = tabla_normalizada.index

fig = px.scatter(
    tabla_normalizada,
    x="Income (Prom)",
    y="Time devoted to leisure and personal care",
    size="Life satisfaction",
    color="Índice de Bienestar",
    hover_name="País",
    title="Equilibrio entre Ingreso y Tiempo Libre",
    labels={
        "Income (Prom)": "Ingreso (normalizado)",
        "Time devoted to leisure and personal care": "Tiempo para ocio y cuidado personal"
    },
    size_max=30
)

fig.show()


### Gráfica Uziel

In [9]:
import plotly.express as px

tabla_normalizada["País"] = tabla_normalizada.index

fig = px.scatter(
    tabla_normalizada,
    x="Income (Prom)",
    y="Life Satisfaction (Prom)",
    hover_name="País",
    title="¿Más ingreso = más felicidad?",
    labels={
        "Income (Prom)": "Ingreso Promedio (normalizado)",
        "Life Satisfaction (Prom)": "Satisfacción con la Vida"
    },
    color="Índice de Bienestar",
    size="Health (Prom)",
    size_max=30
)

fig.show()


## Grafica Alan

In [10]:
import plotly.express as px

# Obtener el top 10 en educación
top_edu = tabla_normalizada.sort_values(by="Education (Prom)", ascending=False).head(10)
top_edu = top_edu.reset_index()

fig = px.bar(
    top_edu,
    x="País",
    y="Education (Prom)",
    title="Top 10 Países con Mejor Nivel Educativo",
    text="Education (Prom)",
    color="Índice de Bienestar"
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()


In [11]:
# Top 10 países donde más seguros se sienten
top_seguridad = tabla_normalizada.sort_values(by="Safety (Prom)", ascending=False).head(10)
top_seguridad = top_seguridad.reset_index()

fig = px.bar(
    top_seguridad,
    x="País",
    y="Safety (Prom)",
    title="Top 10 Países con Mayor Seguridad Percibida",
    text="Safety (Prom)",
    color="Índice de Bienestar"
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()
